In [1]:
using Laplacians
using LinearAlgebra
using SparseArrays

# testing AC

In [2]:
function mod_ldli2Chol(ldli)
    n = length(ldli.colptr)
    m = n + length(ldli.fval)
    li = zeros(Int,m)
    lj = zeros(Int,m)
    lv = zeros(Float64,m)
    lptr = 0

    dhi = zeros(n)
    for i in 1:n
        if ldli.d[i] == 0
            dhi[i] = 1.0
        else
            dhi[i] = sqrt(ldli.d[i])
        end
    end

    scales = ones(n)
    for ii in 1:(n-1)
        i = ldli.col[ii]
        j0 = ldli.colptr[ii]
        j1 = ldli.colptr[ii+1]-1
        scales[i] = prod(1.0 .- ldli.fval[j0:(j1-1)])
    end

    for ii in 1:(n-1)
        i = ldli.col[ii]
        j0 = ldli.colptr[ii]
        j1 = ldli.colptr[ii+1]-1
        scale = scales[i] / dhi[i]

        scj = 1
        for jj in j0:(j1-1)
            j = ldli.rowval[jj]
            f = ldli.fval[jj]

            lptr += 1
            li[lptr] = i
            lj[lptr] = j
            lv[lptr] = -f*scj/scale


            scj = scj*(1-f)
        end
        j = ldli.rowval[j1]

        lptr += 1
        li[lptr] = i
        lj[lptr] = j
        lv[lptr] = -dhi[i]

        lptr += 1
        li[lptr] = i
        lj[lptr] = i
        lv[lptr] = 1/scale

    end

    for i in 1:n
        if ldli.d[i] == 0
            lptr += 1
            li[lptr] = i
            lj[lptr] = i
            lv[lptr] = 0 # rank n-1. This is appropriate for estimating mean LLt, not for applying inverse
            # PREVIOUSLY: lv[lptr] = 1 # NB: rank n because of this. Requires orthog to all-ones
        end
    end

    return sparse(lj,li,lv,n,n) # return lower tri form
    #return sparse(li,lj,lv,n,n)
    #return li, lj, lv
end

mod_ldli2Chol (generic function with 1 method)

In [3]:
n = 9
a = grid2(3,3)

9×9 SparseMatrixCSC{Float64, Int64} with 24 stored entries:
  ⋅   1.0   ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
 1.0   ⋅   1.0   ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
 1.0   ⋅    ⋅    ⋅   1.0   ⋅   1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅   1.0   ⋅   1.0   ⋅   1.0   ⋅ 
  ⋅    ⋅   1.0   ⋅   1.0   ⋅    ⋅    ⋅   1.0
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅   1.0   ⋅   1.0
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅   1.0   ⋅ 

In [4]:
llmat = Laplacians.LLmatp(a)
@time ldli = Laplacians.approxChol(llmat)
lfac = mod_ldli2Chol(ldli)

  0.297205 seconds (370.88 k allocations: 18.492 MiB, 99.71% compilation time)


9×9 SparseMatrixCSC{Float64, Int64} with 25 stored entries:
  1.41421    ⋅     ⋅          ⋅        …    ⋅          ⋅         ⋅ 
 -0.707107  0.0  -0.707107  -0.353553       ⋅          ⋅         ⋅ 
   ⋅         ⋅    1.41421     ⋅             ⋅          ⋅         ⋅ 
 -0.707107   ⋅     ⋅         1.41421      -0.707107    ⋅         ⋅ 
   ⋅         ⋅     ⋅        -0.707107       ⋅        -1.22474    ⋅ 
   ⋅         ⋅   -0.707107    ⋅        …    ⋅          ⋅       -0.707107
   ⋅         ⋅     ⋅          ⋅            1.41421     ⋅         ⋅ 
   ⋅         ⋅     ⋅        -0.353553     -0.707107   1.22474  -0.707107
   ⋅         ⋅     ⋅          ⋅             ⋅          ⋅        1.41421

In [5]:
P = I - ones(n,n)/n

9×9 Matrix{Float64}:
  0.888889  -0.111111  -0.111111  …  -0.111111  -0.111111  -0.111111
 -0.111111   0.888889  -0.111111     -0.111111  -0.111111  -0.111111
 -0.111111  -0.111111   0.888889     -0.111111  -0.111111  -0.111111
 -0.111111  -0.111111  -0.111111     -0.111111  -0.111111  -0.111111
 -0.111111  -0.111111  -0.111111     -0.111111  -0.111111  -0.111111
 -0.111111  -0.111111  -0.111111  …  -0.111111  -0.111111  -0.111111
 -0.111111  -0.111111  -0.111111      0.888889  -0.111111  -0.111111
 -0.111111  -0.111111  -0.111111     -0.111111   0.888889  -0.111111
 -0.111111  -0.111111  -0.111111     -0.111111  -0.111111   0.888889

In [16]:
T = 1000000
ellt = zeros(n,n)
for i = 1:T
    llmat = Laplacians.LLmatp(a)
    ldli = Laplacians.approxChol(llmat)
    lfac = mod_ldli2Chol(ldli)
    ellt += lfac*lfac'
end
ellt = ellt/T

9×9 Matrix{Float64}:
  2.0  -1.0          0.0  -1.0   0.0       0.0   0.0   0.0          0.0
 -1.0   3.0         -1.0   0.0  -1.00027   0.0   0.0   0.00026875   0.0
  0.0  -1.0          2.0   0.0   0.0      -1.0   0.0   0.0          0.0
 -1.0   0.0          0.0   3.0  -1.0       0.0  -1.0   0.0          0.0
  0.0  -1.00027      0.0  -1.0   4.00027  -1.0   0.0  -1.0          0.0
  0.0   0.0         -1.0   0.0  -1.0       3.0   0.0   0.0         -1.0
  0.0   0.0          0.0  -1.0   0.0       0.0   2.0  -1.0          0.0
  0.0   0.00026875   0.0   0.0  -1.0       0.0  -1.0   2.99973     -1.0
  0.0   0.0          0.0   0.0   0.0      -1.0   0.0  -1.0          2.0

In [17]:
norm(lap(a) - ellt)*sqrt(T)

0.6583003683730457

Compare that with AC2

In [64]:
T = 1000000
ellt = zeros(n,n)
for i = 1:T
    llmat = Laplacians.LLmatp(a)
    ldli = Laplacians.approxChol(llmat,2,2)
    lfac = mod_ldli2Chol(ldli)
    ellt += lfac*lfac'
end
ellt = ellt/T

9×9 Matrix{Float64}:
  2.0  -1.0       0.0  -1.0   0.0       0.0   0.0   0.0       0.0
 -1.0   3.0      -1.0   0.0  -1.00003   0.0   0.0   2.95e-5   0.0
  0.0  -1.0       2.0   0.0   0.0      -1.0   0.0   0.0       0.0
 -1.0   0.0       0.0   3.0  -1.0       0.0  -1.0   0.0       0.0
  0.0  -1.00003   0.0  -1.0   4.00003  -1.0   0.0  -1.0       0.0
  0.0   0.0      -1.0   0.0  -1.0       3.0   0.0   0.0      -1.0
  0.0   0.0       0.0  -1.0   0.0       0.0   2.0  -1.0       0.0
  0.0   2.95e-5   0.0   0.0  -1.0       0.0  -1.0   2.99997  -1.0
  0.0   0.0       0.0   0.0   0.0      -1.0   0.0  -1.0       2.0

In [67]:
norm(lap(a) - ellt)*sqrt(T)

0.07225994741213376

In [68]:
T = 1000000
ellt = zeros(n,n)
for i = 1:T
    llmat = Laplacians.LLmatp(a)
    ldli = Laplacians.approxChol(llmat,2,2)
    lfac = mod_ldli2Chol(ldli)
    ellt += lfac*lfac'
end
ellt = ellt/T

9×9 Matrix{Float64}:
  2.0  -1.0           0.0  -1.0   0.0        0.0   0.0   0.0           0.0
 -1.0   3.0          -1.0   0.0  -0.999661   0.0   0.0  -0.000339125   0.0
  0.0  -1.0           2.0   0.0   0.0       -1.0   0.0   0.0           0.0
 -1.0   0.0           0.0   3.0  -1.0        0.0  -1.0   0.0           0.0
  0.0  -0.999661      0.0  -1.0   3.99966   -1.0   0.0  -1.0           0.0
  0.0   0.0          -1.0   0.0  -1.0        3.0   0.0   0.0          -1.0
  0.0   0.0           0.0  -1.0   0.0        0.0   2.0  -1.0           0.0
  0.0  -0.000339125   0.0   0.0  -1.0        0.0  -1.0   3.00034      -1.0
  0.0   0.0           0.0   0.0   0.0       -1.0   0.0  -1.0           2.0